# Scripts for Exporting PyTorch Models to ONNX and CoreML

Depending on the backend, we prefer different qunatization schemes.

- For ONNX we use `uint8` quantization.
- For PyTorch we use `bfloat16` quantization.
- For CoreML we use `float32` representation.

In [ ]:
!pip install --upgrade "uform[torch]" coremltools

In [ ]:
import os
model_name = "unum-cloud/uform-gen2-dpo"
output_directory = "../../"

In [ ]:
import torch
import uform
from PIL import Image
from transformers import AutoModel, AutoProcessor

model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)

prompt = 'Describe the picture'
image = Image.open('../../assets/unum.png')
inputs = processor(text=[prompt], images=[image], return_tensors='pt')

with torch.inference_mode():
     output = model.generate(
        **inputs,
        do_sample=False,
        use_cache=True,
        max_new_tokens=256,
        eos_token_id=151645,
        pad_token_id=processor.tokenizer.pad_token_id
    )
prompt_len = inputs['input_ids'].shape[1]
decoded_text = processor.batch_decode(output[:, prompt_len:])[0]

print(decoded_text)